In [3]:
import numpy as np    #import numpy
from math import factorial as f    #import math

In [4]:
def nCk(n,k):    #define a function to compute the combinations
    """A function to compute n choose k.  The function returns 0 if k>n."""
    try:
        r = f(n)/(f(k)*f(n-k))
    except:
        r = 0.0
    return r


In [5]:
def B(k,n,p):    #define a function to compute the binomial probability mass function
    """A function to compute the probability of k successes given n trials w/ prob p of success."""
    return nCk(n,k)*p**k*(1 - p)**(n - k)


In [6]:
def pmf(k,n_1,p_1,n_2,p_2):    #define a function to compute the derived pmf
    """A function to compute the probability distribution over the number of hits remaining after defending.
    The function takes in:
    k: nmber of hits
    n_1: the number of attacks
    p_1: the likelihood of an attack being successful
    n_2: the number of defenses
    p_2: the likelhood of defending
    The function returns p(k)."""
    p = 0.0
    if k >= 0:
        for i in range(0,n_1):
            p += B(k+i,n_1,p_1)*B(i,n_2,p_2)
    if k < 0:
        for i in range(0,n_2):
            p += B(i,n_1,p_1)*B(k+i,n_2,p_2)
    return p


In [7]:
def cmf(Z,n_1,p_1,n_2,p_2):    #a function to compute the cmf - in this case prob that 1 kills 2
    """A function to compute the probability that computes the CDF of the PMF in the previous cell.
    Substantively, this would be the probability that 1 defeats 2.
    The function takes in:
    Z: the number of hits needed to fell the target
    n_1: the number of attack against the target
    p_1: the likelihood that an attack will be successful
    n_2: the number of defenses
    p_2: the likelihood of defending
    The function returns p(k>=Z)."""
    c = 0.0
    for k in range(Z,n_1+1):
        c += pmf(k,n_1,p_1,n_2,p_2)
    return c


In [8]:
def Cost(shoot,assault,armor,stamina,bulk,n_action,armor_target,stamina_target):
    """A function to compute the cost of a unit given assumptions about the importance of ranged fire vs close combat.
    The function takes in:
    shoot: the number of shooting attacks of the unit to be evaluated
    assault: the number of close combat attacks of the unit to be evaluated
    armor: the armor of the unit to be evaluated
    stamina: the stamina of the unit to be evaluated
    bulk: the number of individuals who can fit in a discrete space
    n_action: the number of actions the unit to be evaluated can make per turn
    armor_target: the armor of the basic target - assume the same for all units to be evaluated
    stamina_target: the stamina of the basic target - assume the same for all units to be evaluated
    The function returns a non-normalized number that represents a weighted combination of the probability that the
    unit to be evaluated kills the basic target and the probability that a unit of the basic target kills the
    unit to be evaluated."""
    #fixed model parameters
    p_hit = 2.0/6.0    #from the dice, the prob of a hit
    p_chit = 1.0/6.0    #from the dice, the prob of a critical hit
    p_defend = 2.0/6.0    #from the dice, the prob of a block

    w_shoot = 0.5    #the relative importance of shooting - must add to one with assault
    w_assault = 0.5    #the relative importance of assault - must add to one with shooting
    w_offense = 0.5    #the relative importance of offense - must add to one with defence
    w_defense = 0.5    #the relative importance of defense - must add to one with offense

    Shoot_base = 6    #the number of base shooting attacks
    Assault_base = 6    #the number of base assault attacks

    hex_size = 3.0    #how much can fit in a hex
    
    c = n_action*(w_offense*(w_shoot*hex_size/bulk*cmf(stamina_target,shoot,p_hit+p_chit,armor_target,p_defend) 
                             + w_assault*hex_size/bulk*cmf(stamina_target,assault,p_hit+p_chit,armor_target,p_defend)) 
                  + w_defense*(w_shoot*(1 - cmf(stamina,Shoot_base,p_hit+p_chit,armor,p_defend)) 
                               + w_assault*(1 - cmf(stamina,Assault_base,p_hit+p_chit,armor,p_defend))))     #compute the cost
    
    return c


In [9]:
#Individual baseline cost with no weapons
Cost_Astartes_base = Cost(0,2,2,2,1,2,2,2)
M = 10/Cost_Astartes_base
print('The normalizing constant is:',M,'.  With this constant an individual basic Astartes is 10 pts.')


The normalizing constant is: 22.677165354330718 .  With this constant an individual basic Astartes is 10 pts.


In [49]:
#special rule costs
Jump_infantry = M*(Cost(0,2,2,2,1,3,2,2) - Cost_Astartes_base)    #assume more actions
Hardened_Armor = M*(Cost(0,2,5,2,1,2,2,2) - Cost_Astartes_base)    #assume more defense - this overcosts
Counterattack = M*(Cost(0,2,2,2,1,2.3,2,2) - Cost_Astartes_base)    #assume more actions
Scout = M*(8.0/6.0*0.5*(Cost(0,2,2,2,1,2.3,2,2) - Cost_Astartes_base))    #assume two extra turns restricted to half of the possible actions
Stealth = M*(Cost(0,2,3,2,1,2,2,2) - Cost_Astartes_base)    #assume more defense - this overcosts
Shrouded = M*(Cost(0,2,4,2,1,2,2,2) - Cost_Astartes_base)    #assume more defense - this overcosts
Slow_and_Purposeful = M*(Cost(0,2,2,2,1,3,2,2) - Cost_Astartes_base)    #assume more actions
Chain_fire = M*(Cost(4,2,2,2,1,3,2,2) + Cost(2,2,2,2,1,3,2,2) + Cost(1,2,2,2,1,3,2,2) - 3.0*Cost_Astartes_base)    #assume more actions
Lone_Killer = M*(Cost(0,2,2,2,1,2,2,2) - Cost(0,2,6,6,1,2,2,2))    #assume a decrease in armor and stamina
Infiltrate = (Scout*2.0)    #four extra turns
Outflank = (Scout*3.0)    #three extra turns
Acute_senses = (0.75*(1.0/3.0*Scout + 1.0/3.0*Infiltrate + 1.0/3.0*Outflank) - Cost_Astartes_base)    #assume fewer enemy actions
Preferred_Enemy = M*(1.0/11.0*(Cost(3,3,2,2,1,3,2,2) - Cost_Astartes_base))    #assume more attacks in shooting and assault
Fury_of_the_Legion = M*(Cost(4,2,2,2,1,1,2,2) + (1 - 5.0/6.0**2.0)*0.5*Cost(2,2,2,2,1,2,2,2) - Cost_Astartes_base)    #equate with larger shoot but fewer actions
Fearless = M*((1 - cmf(2,2,1.0/2.0,2,1.0/3.0))*2.0/3.0)    #equate with likelihood of death in close combat
Sniper = M*(cmf(2,2,1.0/2.0,2,1.0/3.0)*Cost(0,2,4,3,1,2,2,2))    #choose target - probably an officer
Furious_Charge = M*(0.5*(Cost(0,3,2,2,1,2,2,2) - Cost_Astartes_base))    #extra assault dice when in an assault action

Special_rules = {}
Special_rules['Jump_infantry'] = round(Jump_infantry)
Special_rules['Hardened_Armor'] = round(Hardened_Armor)
Special_rules['Counterattack'] = round(Counterattack)
Special_rules['Scout'] = round(Scout)
Special_rules['Stealth'] = round(Stealth)
Special_rules['Shrouded'] = round(Shrouded)
Special_rules['Slow_and_Purposeful'] = round(Slow_and_Purposeful)
Special_rules['Chain_fire'] = round(Chain_fire)
Special_rules['Lone_Killer'] = round(Lone_Killer)
Special_rules['Infiltrate'] = round(Infiltrate)
Special_rules['Outflank'] = round(Outflank)
Special_rules['Acute_senses'] = round(Acute_senses)
Special_rules['Preferred_Enemy'] = round(Preferred_Enemy)
Special_rules['Fury_of_the_Legion'] = round(Fury_of_the_Legion)
Special_rules['Fearless'] = round(Fearless)
Special_rules['Sniper'] = round(Sniper)
Special_rules['Furious_Charge'] = round(Furious_Charge)

{'Acute_senses': 1,
 'Chain_fire': 44,
 'Counterattack': 1,
 'Fearless': 13,
 'Furious_Charge': 3,
 'Fury_of_the_Legion': 9,
 'Hardened_Armor': 6,
 'Infiltrate': 2,
 'Jump_infantry': 5,
 'Lone_Killer': -16,
 'Outflank': 3,
 'Preferred_Enemy': 3,
 'Scout': 1,
 'Shrouded': 4,
 'Slow_and_Purposeful': 5,
 'Sniper': 2,
 'Stealth': 2}

In [158]:
#Costs for individual soldiers
#Headquarters
Centurion = M*Cost(0,2,4,3,1,2,2,2)    #Fearless

#Elites
Veteran = M*Cost(0,3,2,2,1,2,2,2)    #one of Fearless, Sniper, Furious Charge, Outflank, or Tank Hunters Special rule
Destroyer = M*Cost(0,2,2,2,1,2,2,2)    #counterattack special rule
Cataphractii = M*Cost(0,3,5,3,1.5,2,2,2)    #Hardened Armor, slow and purposeful
Tartaros = M*Cost(0,3,5,3,1.5,2,2,2)    

#Troops
Tactical = M*Cost(0,2,2,2,1,2,2,2)    #Fury of the Legion
Recon = M*Cost(0,2,2,2,1,2,2,2)    #Infiltrate, Scout, Outflank, and Acute Senses
Breacher = M*Cost(0,2,3,2,1,2,2,2)    #Hardened Armor
Assault = M*Cost(0,2,2,2,1,2,2,2)    #Jump Infantry

#Support
Support = M*Cost(0,2,2,2,1,2,2,2)
Seeker = M*Cost(0,2,2,2,1,2,2,2)    #Preferred Enemy

Soldiers = {}
Soldiers['Centurion'] = round(Centurion)
Soldiers['Veteran'] = round(Veteran)
Soldiers['Destroyer'] = round(Destroyer)
Soldiers['Cataphractii'] = round(Cataphractii)
Soldiers['Tartaros'] = round(Tartaros)
Soldiers['Tactical'] = round(Tactical)
Soldiers['Recon'] = round(Recon)
Soldiers['Breacher'] = round(Breacher)
Soldiers['Assault'] = round(Assault)
Soldiers['Support'] = round(Support)
Soldiers['Seeker'] = round(Seeker)
Soldiers

{'Assault': 10,
 'Breacher': 12,
 'Cataphractii': 24,
 'Centurion': 20,
 'Destroyer': 10,
 'Recon': 10,
 'Seeker': 10,
 'Support': 10,
 'Tactical': 10,
 'Tartaros': 24,
 'Veteran': 16}

In [183]:
#Shooting Weapon cost factoring in critical effect
Bolter = M*(Cost(2,2,2,2,1,2,2,2) + (1 - 5.0/6.0**2.0)*0.5*Cost(2,2,2,2,1,2,2,2) - Cost_Astartes_base)
Bolt_pistol = Bolter*1.0/2.0
Combibolter = M*(Cost(4,2,2,2,1,2,2,2) + (1.0 - 5.0/6.0**4.0)*0.5*Cost(2,2,2,2,1,2,2,2) - Cost_Astartes_base)
Heavy_bolter = M*(Cost(6,2,2,2,1,2,2,2) + (1 - 5.0/6.0**6.0)*0.5*Cost(2,2,2,2,1,2,2,2) - Cost_Astartes_base)
Meltagun = M*(5.0/6.0**3.0*Cost(3,2,2,2,1,2,2,2) + (1.0 - 5.0/6.0**3.0)*Cost(3,2,2,2,1,2,0,2) - Cost_Astartes_base)
Volkite_caliver = M*(5.0/6.0**4.0*Cost(4,2,2,2,1,2,2,2) + (1.0 - 5.0/6.0**4.0)*Cost(4,2,2,2,1,2,2,0) - Cost_Astartes_base)
Volkite_serpenta = M*(1.0/2.0*(5.0/6.0**3.0*Cost(3,2,2,2,1,2,2,2) + (1.0 - 5.0/6.0**3.0)*Cost(3,2,2,2,1,2,2,0) - Cost_Astartes_base))
Flamer = M*(1.0/2.0*(5.0/6.0**4.0*Cost(4,2,2,2,1,2,2,2) + (1.0 - 5.0/6.0**4.0)*2.0*Cost(4,2,2,2,1,2,2,2) - Cost_Astartes_base))
Hand_flamer = M*(1.0/2.0*(5.0/6.0**3.0*Cost(3,2,2,2,1,2,2,2) + (1.0 - 5.0/6.0**3.0)*2.0*Cost(3,2,2,2,1,2,2,2) - Cost_Astartes_base))
Heavy_flamer = M*(1.0/2.0*(5.0/6.0**6.0*Cost(6,2,2,2,1,2,2,2) + (1.0 - 5.0/6.0**6.0)*2.0*Cost(6,2,2,2,1,2,2,2) - Cost_Astartes_base))
Plasmapistol = M*(1.0/2.0*(5.0/6.0**3.0*Cost(3,2,2,2,1,2,2,2) + (1.0 - 5.0/6.0**3.0)*((B(0.0,4.0,1.0/6.0) + B(1.0,4.0,1.0/6.0))*Cost(7,2,2,2,1,2,2,2) - (B(2.0,4.0,1.0/6.0) + B(3.0,4.0,1.0/6.0) + B(4.0,4.0,1.0/6.0))*Cost(3,2,2,2,1,2,2,2)) - Cost_Astartes_base))
Plasmagun = M*((5.0/6.0**3.0*Cost(3,2,2,2,1,2,2,2) + (1.0 - 5.0/6.0**3.0)*((B(0.0,4.0,1.0/6.0) + B(1.0,4.0,1.0/6.0))*Cost(7,2,2,2,1,2,2,2) - (B(2.0,4.0,1.0/6.0) + B(3.0,4.0,1.0/6.0) + B(4.0,4.0,1.0/6.0))*Cost(3,2,2,2,1,2,2,2)) - Cost_Astartes_base))
Combiweapon = 1.0/2.0*Bolter + 1.0/2.0*(1.0/3.0*Flamer + 1.0/3.0*Plasmagun + 1.0/3.0*Meltagun)
Sniper_rifle = M*(Cost(4,2,2,2,1,2,2,2) + (1 - 5.0/6.0**4.0)*1.0/3.0*1.0/2.0*Cost(2,2,2,2,1,2,2,2) - Cost_Astartes_base)
Missile_launcher = M*(5.0/6.0**5.0*Cost(5,2,2,2,1,2,2,2) + (1.0 - 5.0/6.0**5.0)*Cost(7,2,2,2,1,2,2,0) - Cost_Astartes_base)
Graviton_gun = M*(Cost(6,2,2,2,1,2,2,2) + (1 - 5.0/6.0**6.0)*Cost(2,2,2,2,1,2,2,2) - Cost_Astartes_base)
Autocannon = M*(Cost(6,2,2,2,1,2,2,2) + 0.5*(1 - 5.0/6.0**2.0)*0.5*Cost(2,2,2,2,1,2,2,2) + 0.5*Cost(0,6,2,2,1,2,1,2) - Cost_Astartes_base)

Shooting_weapons = {}
Shooting_weapons['Bolter'] = round(Bolter)
Shooting_weapons['Bolt_pistol'] = round(Bolt_pistol)
Shooting_weapons['Combibolter'] = round(Combibolter)
Shooting_weapons['Heavy_bolter'] = round(Heavy_bolter)
Shooting_weapons['Meltagun'] = round(Meltagun)
Shooting_weapons['Volkite_caliver'] = round(Volkite_caliver)
Shooting_weapons['Volkite_serpenta'] = round(Volkite_serpenta)
Shooting_weapons['Flamer'] = round(Flamer)
Shooting_weapons['Hand_flamer'] = round(Hand_flamer)
Shooting_weapons['Heavy_flamer'] = round(Heavy_flamer)
Shooting_weapons['Plasmapistol'] = round(Plasmapistol)
Shooting_weapons['Plasmagun'] = round(Plasmagun)
Shooting_weapons['Combiweapon'] = round(Combiweapon)
Shooting_weapons['Sniper_rifle'] = round(Sniper_rifle)
Shooting_weapons['Missile_launcher'] = round(Missile_launcher)
Shooting_weapons['Graviton_gun'] = round(Graviton_gun)
Shooting_weapons['Autocannon'] = round(Autocannon)
Shooting_weapons

{'Autocannon': 45,
 'Bolt_pistol': 5,
 'Bolter': 10,
 'Combibolter': 22,
 'Combiweapon': 15,
 'Flamer': 20,
 'Graviton_gun': 38,
 'Hand_flamer': 14,
 'Heavy_bolter': 32,
 'Heavy_flamer': 30,
 'Meltagun': 21,
 'Missile_launcher': 56,
 'Plasmagun': 20,
 'Plasmapistol': 10,
 'Sniper_rifle': 18,
 'Volkite_caliver': 54,
 'Volkite_serpenta': 26}

In [80]:
#melee weapon costs figuring in critical effect
Chain_fist = M*(5.0/6.0**3.0*Cost(0,3,2,2,1,2,2,2) + (1 - 5.0/6.0**3.0)*Cost(0,3,2,2,1,2,2,1) - Cost_Astartes_base)
Power_sword = M*(5.0/6.0**3.0*Cost(0,4,2,2,1,2,2,2) + (1 - 5.0/6.0**3.0)*Cost(0,4,2,2,1,2,1,2) - Cost_Astartes_base)
Chain_sword = M*(5.0/6.0**3.0*Cost(0,3,2,2,1,2,2,2) + (1 - 5.0/6.0**3.0)*Cost(0,4,2,2,1,2,2,2) - Cost_Astartes_base)
Heavy_chain_sword = M*(5.0/6.0**3.0*Cost(0,5,2,2,1,2,2,2) + (1 - 5.0/6.0**3.0)*Cost(0,7,2,2,1,2,2,2) - Cost_Astartes_base)
Thunder_hammer = M*(Cost(0,6,2,2,1,2,2,2) + (1 - 5.0/6.0**3.0)*Cost(0,2,2,2,1,2,2,2) - Cost_Astartes_base)
Power_fist = M*(5.0/6.0**3.0*Cost(0,2,2,2,1,2,2,2) + (1 - 5.0/6.0**3.0)*Cost(0,2,2,2,1,2,0,2) - Cost_Astartes_base)
Lightning_claw_1 = M*(5.0/6.0**3.0*Cost(0,3,2,2,1,2,2,2) + (1 - 5.0/6.0**3.0)*Cost(0,5,2,2,1,2,2,2) - Cost_Astartes_base)
Lightning_claw_2 = M*(5.0/6.0**3.0*Cost(0,5,2,2,1,2,2,2) + (1 - 5.0/6.0**3.0)*Cost(0,9,2,2,1,2,2,2) - Cost_Astartes_base)
Paragon_blade = M*(Cost(0,6,2,2,1,2,2,2) + (1 - 5.0/6.0**3.0)*Cost(0,2,2,2,1,2,1,0) - Cost_Astartes_base)

Melee_weapons = {}
Melee_weapons['Chain_fist'] = round(Chain_fist)
Melee_weapons['Power_sword'] = round(Power_sword)
Melee_weapons['Chain_sword'] = round(Chain_sword)
Melee_weapons['Heavy_chain_sword'] = round(Heavy_chain_sword)
Melee_weapons['Thunder_hammer'] = round(Thunder_hammer)
Melee_weapons['Power_fist'] = round(Power_fist)
Melee_weapons['Lightning_claw_1'] = round(Lightning_claw_1)
Melee_weapons['Lightning_claw_2'] = round(Lightning_claw_2)
Melee_weapons['Paragon_blade'] = round(Paragon_blade)
Melee_weapons

{'Chain_fist': 17,
 'Chain_sword': 11,
 'Heavy_chain_sword': 24,
 'Lightning_claw_1': 17,
 'Lightning_claw_2': 27,
 'Paragon_blade': 57,
 'Power_fist': 5,
 'Power_sword': 15,
 'Thunder_hammer': 31}

In [123]:
#Item costs
Cataphractii_terminator_armor = (M*Cost(0,3,5,3,1.5,2,2,2) + Special_rules['Hardened_Armor'] +
                                   Special_rules['Slow_and_Purposeful'] - M*Cost_Astartes_base)
Tartaros_terminator_armor = (M*Cost(0,3,5,3,1.5,2,2,2) +
                                   Special_rules['Slow_and_Purposeful'] - M*Cost_Astartes_base)
Artificer_armor = M*(Cost(0,2,5,2,1,2,2,2) - Cost_Astartes_base)    #gives 5 defense dice
Iron_halo = Special_rules['Hardened_Armor']    #can reroll three defense dice
Breacher_shield = M*(Cost(0,2,3,2,1,2,2,2) - Cost_Astartes_base)    #gives plus 1 defence die
Frag_grenade = M*(1.0/12.0*(Cost(6,2,2,2,1,2,2,2) - Cost_Astartes_base))    #as on card
Krak_grenade = M*(1.0/12.0*(5.0/6.0**4.0*Cost(4,2,2,2,1,2,2,2) + (1 - 5.0/6.0**4.0)*1.0/3.0*Cost(4,2,2,2,1,2,2,1) - Cost_Astartes_base))    #as on card
Grenade_harness = M*(Cost(0,5,2,2,1,2,2,2) - Cost_Astartes_base)
Phosphex_bombs = M*(1.0/12.0*2*(1 - 5.0/6.0**3.0 - Cost_Astartes_base))    #as on rad grenade
Melta_bombs = M*(5.0/6.0**5.0*Cost(0,5,2,2,1,2,2,2) + (1 - 5.0/6.0**5.0)*1.0/3.0*Cost(0,5,2,2,1,2,0,2) - Cost_Astartes_base)    #as on card
Rad_missile = M*(1 - 5.0/6.0**3.0 - Cost_Astartes_base)    #as on rad grenade
Cameleoline = Special_rules['Stealth']    #more defense
Nuncio_vox = Special_rules['Infiltrate']
Legion_vexilla = M*(Cost(0,2,2,2,1,2,2,2)*(1 - 1.0/6.0**3.0))

Items = {}
Items['Cataphractii_terminator_armor'] = round(Cataphractii_terminator_armor)    #5 armor + hardened armor + slow and purposeful
Items['Tartaros_terminator_armor'] = round(Tartaros_terminator_armor)    #5 armor + slow and purposeful
Items['Artificer_armor'] = round(Artificer_armor)    #5 armor
Items['Iron_halo'] = round(Iron_halo)    #hardened armor
Items['Breacher_shield'] = round(Breacher_shield)    #+1 armor
Items['Frag_grenade'] = round(Frag_grenade)    #see card
Items['Krak_grenade'] = round(Krak_grenade)    #see card
Items['Grenade_harness'] = round(Grenade_harness)    #plus 1 assault dice for each model in target unit
Items['Phosphex_bombs'] = round(Phosphex_bombs)    #as on rad grenade card but for two turns and can move one hex
Items['Melta_bombs'] = round(Melta_bombs)    #see card
Items['Rad_missile'] = round(Rad_missile)    #as on rad grenade card
Items['Cameleoline'] = round(Cameleoline)    #give stealth
Items['Nuncio_vox'] = round(Nuncio_vox)    #allows deep strike
Items['Legion_vexilla'] = round(Legion_vexilla)    #as in rule book
Items

{'Artificer_armor': 6,
 'Breacher_shield': 2,
 'Cameleoline': 2,
 'Cataphractii_terminator_armor': 25,
 'Frag_grenade': 2,
 'Grenade_harness': 17,
 'Iron_halo': 6,
 'Krak_grenade': 0,
 'Legion_vexilla': 10,
 'Melta_bombs': 1,
 'Nuncio_vox': 2,
 'Phosphex_bombs': 2,
 'Rad_missile': 12,
 'Tartaros_terminator_armor': 19}

In [134]:
Moritat = Shooting_weapons['Bolt_pistol'] + Items['Phosphex_bombs'] + Special_rules['Scout'] + Special_rules['Counterattack'] + Special_rules['Chain_fire'] + Special_rules['Lone_Killer']
Champion = M*(Cost(0,6,2,2,1,2,2,2) - Cost_Astartes_base) + Special_rules['Fearless']
Vigilator = Special_rules['Scout'] + Special_rules['Sniper'] + Special_rules['Infiltrate'] + Shooting_weapons['Combiweapon'] + Items['Cameleoline']
Consul_roles = {}
Consul_roles['Moritat'] = round(Moritat)
Consul_roles['Champion'] = round(Champion)
Consul_roles['Vigilator'] = round(Vigilator)
Consul_roles

{'Champion': 34, 'Moritat': 37, 'Vigilator': 29}

In [174]:
#Squad costs
Squads = {}

#Headquarters
Squads['Centurion'] = Soldiers['Centurion'] + Shooting_weapons['Bolt_pistol'] + Melee_weapons['Chain_sword'] + Items['Frag_grenade'] + Items['Krak_grenade']
Squads['Centurion_no_weapons'] = Soldiers['Centurion'] + Items['Frag_grenade'] + Items['Krak_grenade']

#Elites
Squads['Destroyer'] = (5*(Soldiers['Destroyer'] + Special_rules['Counterattack'] +
                          2*Shooting_weapons['Bolt_pistol'] + Melee_weapons['Chain_sword']) + Items['Phosphex_bombs'])
Squads['Destroyer_no_weapons'] = (5*(Soldiers['Destroyer'] + Special_rules['Counterattack']) + Items['Phosphex_bombs'])

Squads['Veterans'] = (5*(Soldiers['Veteran'] + Shooting_weapons['Bolter'] + Shooting_weapons['Bolt_pistol'] +
                        Melee_weapons['Chain_sword'] + 
                       max(Special_rules['Fearless'],Special_rules['Sniper'],Special_rules['Furious_Charge'], 
                           Special_rules['Outflank'])) + Items['Frag_grenade'] + Items['Krak_grenade'])
Squads['Veterans_no_weapons'] = (5*(Soldiers['Veteran'] +
                       max(Special_rules['Fearless'],Special_rules['Sniper'],Special_rules['Furious_Charge'], 
                           Special_rules['Outflank'])) + Items['Frag_grenade'] + Items['Krak_grenade'])

Squads['Cataphractii'] = 5*(Soldiers['Cataphractii'] + Special_rules['Hardened_Armor'] +
                            Special_rules['Slow_and_Purposeful'] + Shooting_weapons['Combibolter'] + Melee_weapons['Power_sword'])
Squads['Cataphractii_no_weapons'] = 5*(Soldiers['Cataphractii'] + Special_rules['Hardened_Armor'] +
                            Special_rules['Slow_and_Purposeful'])

Squads['Tartaros'] = 5*(Soldiers['Tartaros'] + 
                            Special_rules['Slow_and_Purposeful'] + Shooting_weapons['Combibolter'] + Melee_weapons['Power_sword'])
Squads['Tartaros_no_weapons'] = 5*(Soldiers['Tartaros'] + 
                            Special_rules['Slow_and_Purposeful'])

#Troops
Squads['Tactical'] = 10*(Soldiers['Tactical'] + Special_rules['Fury_of_the_Legion'] +
                         Shooting_weapons['Bolter']) + Items['Frag_grenade'] + Items['Krak_grenade']
Squads['Tactical_no_weapons'] = 10*(Soldiers['Tactical'] + Special_rules['Fury_of_the_Legion']) + Items['Frag_grenade'] + Items['Krak_grenade']

Squads['Breacher'] = 10*(Soldiers['Breacher'] + Special_rules['Hardened_Armor'] 
                         + Special_rules['Fury_of_the_Legion'] + Shooting_weapons['Bolter'] + 
                         Items['Breacher_shield']) + Items['Frag_grenade'] + Items['Krak_grenade']
Squads['Breacher_no_weapons'] = 10*(Soldiers['Breacher'] + Special_rules['Hardened_Armor'] 
                         + Special_rules['Fury_of_the_Legion'] +
                         Items['Breacher_shield']) + Items['Frag_grenade'] + Items['Krak_grenade']

Squads['Recon'] = 5*(Soldiers['Recon'] + max(Special_rules['Infiltrate'] + Special_rules['Scout'],
                                             Special_rules['Outflank'],Special_rules['Acute_senses'])
                    + Shooting_weapons['Bolter'])
Squads['Recon_no_weapons'] = 5*(Soldiers['Recon'] + max(Special_rules['Infiltrate'] + Special_rules['Scout'],
                                             Special_rules['Outflank'],Special_rules['Acute_senses']))

Squads['Assault'] = (10*(Soldiers['Assault'] + Shooting_weapons['Bolt_pistol'] + 
                        Melee_weapons['Chain_sword'] + Special_rules['Jump_infantry']) +
                     Items['Frag_grenade'] + Items['Krak_grenade'])
Squads['Assault_no_weapons'] = (10*(Soldiers['Assault'] + Special_rules['Jump_infantry']) +
                     Items['Frag_grenade'] + Items['Krak_grenade'])

#Support
Squads['Support'] = 5*(Soldiers['Support'] + Shooting_weapons['Flamer'])
Squads['Support_no_weapons'] = 5*(Soldiers['Support'])

Squads['Seeker'] = (5*(Soldiers['Seeker'] + Shooting_weapons['Bolter'] + Shooting_weapons['Bolt_pistol']) + 
                     Items['Frag_grenade'] + Items['Krak_grenade'])
Squads['Seeker_no_weapons'] = (5*(Soldiers['Seeker']) + 
                     Items['Frag_grenade'] + Items['Krak_grenade'])


Squads

{'Assault': 312,
 'Assault_no_weapons': 152,
 'Breacher': 392,
 'Breacher_no_weapons': 292,
 'Cataphractii': 360,
 'Cataphractii_no_weapons': 175,
 'Centurion': 38,
 'Centurion_no_weapons': 22,
 'Destroyer': 162,
 'Destroyer_no_weapons': 57,
 'Recon': 115,
 'Recon_no_weapons': 65,
 'Seeker': 127,
 'Seeker_no_weapons': 52,
 'Support': 150,
 'Support_no_weapons': 50,
 'Tactical': 292,
 'Tactical_no_weapons': 192,
 'Tartaros': 330,
 'Tartaros_no_weapons': 145,
 'Veterans': 277,
 'Veterans_no_weapons': 147}